# 环境初始化

In [1]:
# 初始化Python科学计算环境
%pylab
# %matplotlib inline
import pandas as pd
import seaborn as sns
import scipy.io as sio

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# 其他的一些库及初始环境
import os

data_dir = u'D:\Documents\工作存档\数学建模\\2016美赛\ProblemCDATA'
report_dir = u'D:\Projects\Tex\MCM_2016\MCM_ICM\数据\\report'

work_dir = os.path.realpath(os.curdir)
mat_dir = os.path.join(work_dir, 'mat')

if not os.path.exists(report_dir):
    os.mkdir(report_dir)
if not os.path.exists(mat_dir):
    os.mkdir(mat_dir)

work_dir

'D:\\Projects\\PyCharm\\math_modeling_learning\\mcm'

In [3]:
# 初始化Matlab环境
import matlab.engine
eng = matlab.engine.start_matlab()
eng.userpath(os.path.realpath(os.path.curdir), nargout=0)
# eng.doc(nargout=0)
eng.userpath()

'D:\\Projects\\PyCharm\\math_modeling_learning\\mcm;'

# 数据处理
1. 所有分类的数据全部导出到 report_dir
2. 只有在参与计算时才临时填充缺失值

In [4]:
# 候选学校名单
# 全部为 IPED 记录的所机构, 共 2977 所
candidate_schools = pd.read_csv(
    os.path.join(data_dir, r'IPEDS UID for Potential Candidate Schools.csv'),
    skipinitialspace=True,
    index_col=0,
    dtype={
        'UNITID':np.int
    }
)

# 共有 2977 所候选学校
candidate_schools.shape

(2977, 3)

In [5]:
# 评价数据
scorecard_elements = pd.read_csv(
    os.path.join(data_dir, r'Most+Recent+Cohorts+(Scorecard+Elements).csv'),
    skipinitialspace=True,
    index_col=0,
    na_values=['NULL', 'PrivacySuppressed'],
    # usecols=[
    #     0,  # UNITID 
    #     3,  # INSTNM
    #     4,  # CITY
    #     5,  # STABBR
    # 
    # ]
    dtype={
        'UNITID':np.int
    }
)

# 筛选出候选名单的数据
# 41 所候选学校缺少数据
scorecard_elements = scorecard_elements.ix[candidate_schools.index]

# 只选取在运转的学校
# 8 所学校不在运转
scorecard_elements = scorecard_elements[scorecard_elements['CURROPER'] == 1]

# 剩余 2977 - 41 -8 = 2928 所学校
# 除去作为索引的字段, 共有 121 个字段
scorecard_elements.shape

(2928, 121)

In [6]:
# 评价指标描述文件
data_dictionary = pd.read_csv(
    os.path.join(data_dir, r'CollegeScorecardDataDictionary-09-08-2015.csv'),
    skipinitialspace=True,
    # index_col=0,
    # na_values='NULL'
    usecols=[
        0,  # NAME OF DATA ELEMENT
        1,  # dev-category
        #  2, # developer-friendly name
        3,  # API data type
        4,  # VARIABLE NAME
        5,  # VALUE
        6,  # LABEL
        7,  # SOURCE
        # 8  # NOTES
    ],
    # dtype={
    #     'VALUE':np.int
    # }
)

keys = [
    'NAME OF DATA ELEMENT',
    'dev-category',
    'API data type',
    'VARIABLE NAME',
    'SOURCE'
]

# 填充枚举属性的空白值
map(lambda v: data_dictionary[v].fillna(method='ffill', inplace=True), keys)

# VALUE 字段有些地方有空格
# data_dictionary['VALUE'].str.strip()

# 只保留我们用到的指标
data_dictionary = pd.concat(
    [data_dictionary[data_dictionary['VARIABLE NAME'] == v] for v in scorecard_elements.keys()])
data_dictionary.to_csv(os.path.join(report_dir, 'CollegeScorecardDataDictionary.csv'))

# 将枚举值进行填充后行数增加到 213 行
data_dictionary.shape

(213, 7)

In [7]:
# 合并刚才因填充后重复的行来达到区分枚举字段的作用
grouped_data_dictionary = data_dictionary.groupby(keys)
value_counts = grouped_data_dictionary['VALUE'].count()
# 枚举类型的数据
em_index = value_counts[value_counts > 0]
em_index.to_csv(os.path.join(report_dir, 'EnumCollegeScorecardElements.csv'))
# 非枚举类型的数据
unem_index = value_counts[value_counts <= 0]
unem_index = unem_index.sort_index(level=1)
unem_index.to_csv(os.path.join(report_dir, 'UnenumCollegeScorecardElements.csv'))
# 15 个枚举字段, 106个非枚举字段, 共 121 个字段, 与原数据一致

len(em_index), len(unem_index)

(15, 106)

In [8]:
# 提取出字段
not_number_elements = em_index.index.get_level_values(3).tolist()
# 添加算法遗漏的非数值字段
not_number_elements.extend([
    'opeid6',
    'OPEID',
    'CITY',
    'INSTNM',
    'HCM2',
    'STABBR',
    'INSTURL',
    'NPCURL'
])

# 121 - 15 - 8 = 98 个数值字段
number_elements_index = scorecard_elements.keys().difference(not_number_elements)
# 只包含数值字段的数据
number_elements = scorecard_elements[number_elements_index]
# 导出方便计算
number_elements.to_csv(os.path.join(report_dir, 'NumberElements.csv'))

number_elements.shape

(2928, 98)

In [9]:
# 四年制学校
four_year_institutions = number_elements[
    np.logical_or(
        np.logical_or(
            number_elements['RET_FT4'].notnull(),
            number_elements['RET_PT4'].notnull()
        ),
        number_elements['C150_4_POOLED_SUPP'].notnull()
    )]
# 删除没有数据的三列
# 这些数据是 number_elements 的切片, 不能就地删除
four_year_institutions = four_year_institutions.drop(['RET_FTL4',
                                                      'RET_PTL4',
                                                      'C200_L4_POOLED_SUPP'],
                                                     axis=1)
four_year_institutions.to_csv(os.path.join(report_dir, 'four_year_institutions.csv'))
# 低于四年制学校
less_four_year_institutions = number_elements[
    np.logical_or(
        np.logical_or(
            number_elements['RET_FTL4'].notnull(),
            number_elements['RET_PTL4'].notnull()
        ),
        number_elements['C200_L4_POOLED_SUPP'].notnull()
    )]
# 删除没有数据的三列
less_four_year_institutions = less_four_year_institutions.drop(['RET_FT4',
                                                                'RET_PT4',
                                                                'C150_4_POOLED_SUPP'],
                                                               axis=1)
less_four_year_institutions.to_csv(os.path.join(report_dir, 'less_four_year_institutions.csv'))
# 未知年制的学校
known_year_institutions = number_elements[
    np.logical_and(
        np.logical_and(
            np.logical_and(
                number_elements['RET_FT4'].isnull(),
                number_elements['RET_PT4'].isnull()
            ),
            np.logical_and(
                number_elements['RET_FTL4'].isnull(),
                number_elements['RET_PTL4'].isnull()
            )
        ),
        np.logical_and(
            number_elements['C150_4_POOLED_SUPP'].isnull(),
            number_elements['C200_L4_POOLED_SUPP'].isnull()
        )
    )]

four_year_institutions.shape[0], less_four_year_institutions.shape[0], known_year_institutions.shape[0]

(1896, 1007, 25)

# 数值计算

In [10]:
# 使用各字段的平均值填充缺失值
fyi_filled = four_year_institutions.fillna(four_year_institutions.mean())
lfyi_filled = less_four_year_institutions.fillna(less_four_year_institutions.mean())

# ROI 模型
$$ROI = \frac{I - M}{M}$$
$$I = n  \cdot [k \cdot Salary + (1-k) \cdot m]$$
$$n = s \cdot g \cdot [p \cdot l_1 + (1-p) \cdot l_2]$$

- $I$ 表示毕业学生收入总和；
- $n$ 表示毕业学生人数；
- $k$ 表示6年后收入达到阈值的人数比例（gt\_25k\_p6）；
- $m$ 为毕业学生收入中位数（md\_earn\_wne\_p10）；
- $s$ 表示招生人数（UGDS）；
- $l_1$ 表示兼职留校率（RET\_PT4/RET\_PTL4）；
- $l_2$ 表示非兼职留校率（RET\_FT4/RET\_FTL4）；
- $p$ 表示兼职率（PPTUG_EF）；
- $g$ 表示学校毕业率（C150\_4\_POOLED\_SUPP/C200\_L4\_POOLED\_SUPP）；
- $Salary$ 为学生收入，为常数25000
- $M$ 表示一年内投资金额；

In [11]:
# 学位授予比例
PCIP_index = [
    'PCIP01', 'PCIP03', 'PCIP04', 'PCIP05', 'PCIP09', 'PCIP10', 'PCIP11', 'PCIP12',
    'PCIP13', 'PCIP14', 'PCIP15', 'PCIP16', 'PCIP19', 'PCIP22', 'PCIP23', 'PCIP24',
    'PCIP25', 'PCIP26', 'PCIP27', 'PCIP29', 'PCIP30', 'PCIP31', 'PCIP38', 'PCIP39',
    'PCIP40', 'PCIP41', 'PCIP42', 'PCIP43', 'PCIP44', 'PCIP45', 'PCIP46', 'PCIP47',
    'PCIP48', 'PCIP49', 'PCIP50', 'PCIP51', 'PCIP52', 'PCIP54'
]
# 学位授予比例和
fyi_filled[PCIP_index].sum(axis=1).describe()

count    1896.000000
mean        0.997356
std         0.051299
min         0.000000
25%         0.999900
50%         1.000000
75%         1.000100
max         1.000700
dtype: float64

In [32]:
# ROI = 1/M * I - 1
def cal_I(df, is_4_year, M=None):
    Salary = 25000
    s = df['UGDS']
    p = df['PPTUG_EF']
    k = df['gt_25k_p6']
    m = df['md_earn_wne_p10']
    if is_4_year:
        l1 = df['RET_PT4']
        l2 = df['RET_FT4']
        g = df['C150_4_POOLED_SUPP']
    else:
        l1 = df['RET_PTL4']
        l2 = df['RET_FTL4']
        g = df['C200_L4_POOLED_SUPP']
    n = s * g * (p * l1 + (1 - p) * l2)
    I = n * (k * Salary + (1 - k) * m)
    I.sort_values(ascending=False, inplace=True)
    return I


def export_i_rank(filename, I, is_4_year):
    I_rank = scorecard_elements.ix[I.index]
    I_rank['I'] = I
    I_rank['4-year-institution'] = is_4_year
    I_rank.to_csv(os.path.join(report_dir, '.'.join([filename, 'csv'])))
    return I_rank

# 四年制学校排名
fyi_I = cal_I(fyi_filled, True)
fyi_I_rank = export_i_rank('fyi_I_rank', fyi_I, True)
# # 非四年制排名
lfyi_I = cal_I(lfyi_filled, False)
lfyi_I_rank = export_i_rank('lfyi_I_rank', lfyi_I, False)
# 总排名
I_rank = pd.concat([fyi_I_rank, lfyi_I_rank]).sort_values('I', ascending=False)
I_rank.to_csv(os.path.join(report_dir, 'I_rank.csv'))

# I_rank[['INSTNM', 'CITY', 'I']].head()

,INSTNM,CITY,I
UNITID,,,
204796,Ohio State University-Main Campus,Columbus,1.004124e+09
228723,Texas A & M University-College Station,College Station,9.787565e+08
214777,Pennsylvania State University-Main Campus,University Park,9.744104e+08
228778,The University of Texas at Austin,Austin,9.052115e+08
132903,University of Central Florida,Orlando,8.697867e+08


In [13]:
# 贷款率,佩尔助学金,毕业生的借款中位数,毕业生的借款中位数(10年的每月付款)与I的相关性
correlation_with_rank = I_rank[['PCTFLOAN', 'PCTPELL','GRAD_DEBT_MDN_SUPP','GRAD_DEBT_MDN10YR_SUPP']]
# 使用平均值填补缺失值
correlation_with_rank = correlation_with_rank.fillna(correlation_with_rank.mean())
x = np.arange(correlation_with_rank.shape[0])
line_kws=dict(color='g')
plt.subplot(221)
sns.regplot(x, 'PCTPELL', correlation_with_rank, line_kws=line_kws)
plt.subplot(222)
sns.regplot(x, 'PCTFLOAN', correlation_with_rank, line_kws=line_kws)
plt.subplot(223)
sns.regplot(x, 'GRAD_DEBT_MDN_SUPP', correlation_with_rank, line_kws=line_kws)
plt.subplot(224)
sns.regplot(x, 'GRAD_DEBT_MDN10YR_SUPP', correlation_with_rank, line_kws=line_kws)

show()

In [51]:
# 仅有佩尔助学金与 I 相关性较强
# 填充缺失值
PETPELL = I_rank['PCTPELL'].fillna(I_rank['PCTPELL'].mean())
I_rank['I*p'] = I_rank['I'] * PETPELL
I_rank['cvc'] = I_rank['I'].cumsum() / I_rank['I'].sum()
for i in np.arange(0.1, 1.1, 0.1):
    print i, I_rank[I_rank['cvc'] <= i].shape[0] ,'|',i, I_rank_with_p[I_rank_with_p['cvc'] <= i].shape[0]

# %matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111)
step = 100
x = np.arange(1, I_rank.shape[0]+ 1)
# ax.set_ylim()
y = I_rank['cvc']
ax.plot(x,y)
for x_i in np.arange(step, 1000, step):
    y_i = I_rank['I'].values[x_i]
    y_max = y_i/plt.ylim()[1]
    ax.axvline(x_i, color='r', ymax = y_max)
    ax.annotate(I_rank_with_p['cvc'].values[x_i], (x_i, y_i * 1.2), textcoords='data')
    
I_rank_with_p = I_rank.sort_values('I*p', ascending=False)
I_rank_with_p['cvc'] = I_rank_with_p['I*p'].cumsum()/I_rank_with_p['I*p'].sum()
26 * 110  + 24 * 60
I_rank.head(50)['I']

I_rank[['INSTNM']].head(200).to_excel(os.path.join(report_dir, u'造假.xls'))

0.1 16 | 0.1 17
0.2 42 | 0.2 45
0.3 79 | 0.3 88
0.4 130 | 0.4 146
0.5 206 | 0.5 234
0.6 317 | 0.6 363
0.7 481 | 0.7 561
0.8 744 | 0.8 870
0.9 1214 | 0.9 1381
1.0 2903 | 1.0 2903


In [40]:
plot(np.arange(I_rank_with_p['cvc'].shape[0]),I_rank_with_p['cvc'])

In [17]:
# 筛选出来的 16 个重要字段
important_elemens_index = [
    'UGDS',
    'PPTUG_EF',
    'RET_FT4',
    'RET_FTL4',
    'RET_PT4',
    'RET_PTL4',
    'PCTFLOAN',
    'UG25abv',
    'GRAD_DEBT_MDN_SUPP',
    'GRAD_DEBT_MDN10YR_SUPP',
    'RPY_3YR_RT_SUPP',
    'C150_4_POOLED_SUPP',
    'C200_L4_POOLED_SUPP',
    'md_earn_wne_p10',
    'gt_25k_p6',
    'PCTPELL'
]

# 重要字段的数据
important_elemens = number_elements[important_elemens_index]
important_elemens.to_excel(os.path.join(report_dir, 'important_elements.xls'))

important_elemens.shape

(2928, 16)

# 基于 PSO 投资风险模型

## 风险因子 e 的范围

In [60]:
a = np.random.random(size=16)
a = a / np.sum(a)
np.sort(a)

array([ 0.02502804,  0.02620487,  0.02860422,  0.03624605,  0.03740682,
        0.04034374,  0.04446342,  0.04730842,  0.07110265,  0.0794297 ,
        0.08136016,  0.08250472,  0.08735481,  0.09386824,  0.10061932,
        0.11815481])